In [ ]:
%%javascript

# Bind cmd-shift-d to duplicate line of code
Jupyter.keyboard_manager.edit_shortcuts.add_shortcut('cmd-shift-d', {
    help : 'Duplicate current line',
    help_index : 'zz',
    handler: function(env) {
        var cm=env.notebook.get_selected_cell().code_mirror;
        // get a position of a current cursor in a current cell
        var current_cursor = cm.doc.getCursor();

        // read a content from a line where is the current cursor
        var line_content = cm.doc.getLine(current_cursor.line);
        cm.execCommand('goLineEnd');
        cm.execCommand('newlineAndIndent');
        cm.execCommand('indentLess');
        cm.doc.replaceSelection(line_content);
        cm.doc.setCursor(current_cursor.line + 1, current_cursor.ch);
        return false;
    }}
);

__Universal Robot Control__

In [ ]:
import urx
import time, pickle
import numpy as np
import math3d as m3d

In [ ]:
robot = urx.Robot("10.42.0.162", use_rt=True)

In [ ]:
robot.set_digital_out(0, 1)
time.sleep(1)
robot.set_digital_out(0, 0)

In [ ]:
robot.set_tcp((0, 0, 0.05, 0, 0, 0)) #ckeck Installation tab

In [ ]:
robot.translate((0, 0, 0.05), acc=0.05, vel=0.05) #acceleration, velocity

In [ ]:
robot.translate_tool((0, 0, -0.03), acc=0.05, vel=0.05)

In [ ]:
robot.get_pose()

In [ ]:
robot.get_pos()

In [ ]:
robot.getl()

In [ ]:
def save_current_pos(fname):
    p = robot.getl()
    np.savetxt(fname, np.array(p))
       

def move_to_pos(fname, *args, **kwargs):
    p = np.loadtxt(fname)
    robot.movel(p, *args, **kwargs)
    
def print_pos(fname):
    p = np.loadtxt(fname)
    print(p)
    

In [ ]:
p = np.array(robot.getl())
p[0] = x_robot
p[1] = y_robot
robot.movel(p, vel=0.05, acc=0.05)

In [ ]:
move_to_pos("in", vel=0.05, acc=0.05)

In [ ]:
robot.getl()

In [ ]:
robot.x += 0.01 #1 hour 10 minutes up to this point

In [ ]:
p = robot.get_pos()

In [ ]:
p.array

In [ ]:
m3d.Vector(p.array)

In [ ]:
import threading
from IPython.display import display
import ipywidgets as widgets
import time

fw = widgets.HTML(
    value='',
    placeholder='No data',
    description='Force:',
)

def observe_force(fw):
    while True:
        fw.value = "<br>".join(["{0:5} {1:7.3f}".format(*c) for c in 
                                zip("x y z rx ry rz".split(), robot.get_tcp_force())])

thread = threading.Thread(target=observe_force, args=(fw,))
display(fw)
thread.start()



In [ ]:
robot.get_tcp_force().__str__()

__Here starts CV__

In [ ]:
%matplotlib notebook

import cv2 #pip install opencv-python
import matplotlib.pyplot as plt
import signal, time
import numpy as np
from numpy import linalg
#to check: cv2.getBuildInformation()
print (*filter(lambda s: "FFMPEG" in s, cv2.getBuildInformation().split("\n")))
#for opencv with ffmpeg in conda
#https://github.com/conda-forge/opencv-feedstock/
#pip uninstall opencv
#pip uninstall opencv-python
#conda unistall opencv
#conda install conda=4.0.11
#conda config --add channels conda-forge
#conda install opencv


In [ ]:
def signal_handler(signal, frame):
    # KeyboardInterrupt detected, exiting
    global is_interrupted
    is_interrupted = True

In [ ]:
%%bash
ls /dev | grep video

In [ ]:
vc = cv2.VideoCapture(1) #0 for the first webcam, 1 for the second..

In [ ]:
def imshow(frame, from_color_space='bgr'):
    plt.figure()
    if from_color_space == 'bgr':
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)    # makes the blues image look real colored
    elif from_color_space == 'hsv':
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_HSV2RGB)    # makes the blues image look real colored
    else:
        rgb_frame=frame
    webcam_preview = plt.imshow(rgb_frame)

def online_view():
    plt.ion()

    if vc.isOpened(): # try to get the first frame
        is_capturing, frame = vc.read()
        imshow(frame)   
    else:
        is_capturing = False

    signal.signal(signal.SIGINT, signal_handler)
    is_interrupted = False
    while is_capturing:
        is_capturing, frame = vc.read()
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)    # makes the blues image look real colored
        webcam_preview.set_data(frame)
        plt.draw()

        try:    # Avoids a NotImplementedError caused by `plt.pause`
            plt.pause(0.05)
        except Exception:
            pass
        if is_interrupted:
            vc.release()
            break
    
def get_frame():
    timeout = time.time() + 0.1
    while time.time() < timeout:
        if vc.isOpened(): # try to get the first frame
            is_capturing, frame = vc.read()
        else:
            raise (Exception("Unable to capture"))
    return frame


#frame = get_frame()
#imshow(frame)

#online_view()

In [ ]:
frame = get_frame()
imshow(frame)

__The blob detection__

In [ ]:
move_to_pos('in', acc=0.05, vel=0.05)

In [ ]:
# Read image
# im = cv2.imread("detection/cubes_4.png")
im = get_frame()
cv2.imwrite('test.png', im)

imshow(im)

# Blur image to remove noise
im = cv2.GaussianBlur(im, (5,5), 40)

# imshow(im)

hsv = cv2.cvtColor(im, cv2.COLOR_BGR2HSV)

greenMin = (30, 90, 120) #./range-detector.py -i test.png -f HSV
greenMax = (150, 255, 255)

# Sets pixels to white if in purple range, else will be set to black
mask = cv2.inRange(hsv, greenMin, greenMax)

# Bitwise-AND of mask and purple only image - only used for display
masked = cv2.bitwise_and(im, im, mask= mask)

imshow(masked)

# dilate makes the in range areas larger
mask = cv2.erode(mask, None, iterations=5)

# mask = cv2.dilate(mask, None, iterations=8)


# Bitwise-AND of mask and purple only image - only used for display
masked = cv2.bitwise_and(im, im, mask= mask)

imshow(masked)

In [ ]:
pts = []

# find contours in the mask and initialize the current
# (x, y) center of the ball
cnts = cv2.findContours(mask.copy(), cv2.RETR_LIST,
    cv2.CHAIN_APPROX_SIMPLE)[-2]
center = None

print (len(cnts))

# only proceed if at least one contour was found
for c in cnts:
    # find the largest contour in the mask, then use
    # it to compute the minimum enclosing circle and
    # centroid
#     c = max(cnts, key=cv2.contourArea)
    ((x, y), radius) = cv2.minEnclosingCircle(c)

    # only proceed if the radius meets a minimum size
    if radius > 25 and radius < 200:
        print (radius)
        
        M = cv2.moments(c)
        center = (int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"]))
        if center[1] > 400:
            continue
        # draw the circle and centroid on the frame,
        # then update the list of tracked points
        cv2.circle(im, (int(x), int(y)), int(radius),
            (0, 255, 255), 2)
        cv2.circle(im, center, 5, (0, 0, 255), -1)

        # update the points queue
        pts.append(center)

imshow(im)

In [ ]:
pts

In [ ]:
# save_current_pos('green')

In [ ]:
move_to_pos('in', acc=0.05, vel=0.05)

In [ ]:
# save_current_pos('white')